In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# Enter foldername
FOLDERNAME = 'Shared drives/DeepDoughnut'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Ensures Python interpreter of Colab VM can load python files.
import sys
sys.path.append('/content/drive/Shared drives/{}'.format(FOLDERNAME))

#Set your path
path='/content/drive/Shared drives/DeepDoughnut'


Mounted at /content/drive


In [2]:
import os
os.chdir(path)

In [8]:
! ls -1q new_folder | wc -l 

3619


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
from scipy.io import loadmat

In [ ]:
def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)

In [ ]:
import zipfile
from PIL import Image
import numpy as np

X = []
y_label = []

zip_folders=['part1.zip', 'part2.zip', 'part3.zip', 'part4.zip', 'part10.zip']

for zip_folder in zip_folders:
  file_name=os.path.join(path, 'UCF_image_data', zip_folder)
  with zipfile.ZipFile(file_name, "r") as zip_data:
    content_list = zip_data.namelist()
    for i in range(1, len(content_list), 30):
        name_file=content_list[i]
        img_bytes = zip_data.open(name_file)          # 1
        img_data = Image.open(img_bytes).resize((32, 32), resample=0)             # 2
        # ndarray with shape=(1024, 1280, 3)
        image_as_array = np.array(img_data, np.uint8) # 3
        #image_reshaped = np.array([rebin(image_as_array[:, :, 0], (32, 32)), rebin(image_as_array[:, :, 1], (32, 32)), rebin(image_as_array[:, :, 2], (32, 32))])
        #image_reshaped=image_reshaped.transpose([1, 2, 0])
        X.append(image_as_array)
        y_label.append(int(name_file.split('_')[0]))

In [ ]:
X=np.array(X)
print(X.shape)
X = np.reshape(X, (X.shape[0], -1))

(969, 32, 32, 3)


In [ ]:
X.shape

(969, 3072)

In [ ]:
#get labels from y_lab
labels = pd.read_csv(os.path.join(path, 'output_data/ucf_lat_lon_geocoded_with_outcomes.csv'))

In [ ]:
labels['change_in_growth_rate']=labels['post_pct_change']-labels['pre_pct_change']
labels.set_index('merge_col')
labels['label_discrete']=labels.change_in_growth_rate.astype(str).str[:4]
labels['label_continuous']=labels.change_in_growth_rate

KNN with discrete labels

In [ ]:
X_train.shape

(581, 3072)

In [ ]:
X_test.shape

(194, 3072)

In [ ]:
from sklearn.model_selection import train_test_split
#.6, .2, .2
y=labels.loc[y_label, 'label_discrete']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) 

In [ ]:
# Create a kNN classifier instance. 
from sklearn.neighbors import KNeighborsClassifier
# train and evaluate a k-NN classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier()
model.fit(X_train, y_train)
acc = model.score(X_val, y_val)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 54.64%


In [ ]:
pd.Series(model.predict(X_val)).value_counts()

5.44    119
0.56     40
10.5     26
2.69      8
-2.8      1
dtype: int64

In [ ]:
y_val.value_counts()

5.44    104
0.56     57
2.69     11
10.5      9
1.23      5
5.06      3
-2.8      3
-0.3      2
Name: label_discrete, dtype: int64

In [ ]:
#Create an SVM classifier
from sklearn.svm import SVC
# train and evaluate an SVM
print("[INFO] evaluating raw pixel accuracy...")
model = SVC(class_weight='balanced')
model.fit(X_train, y_train)
acc = model.score(X_val, y_val)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 64.95%


In [ ]:
y_val.value_counts()

5.44    104
0.56     57
2.69     11
10.5      9
1.23      5
5.06      3
-2.8      3
-0.3      2
Name: label_discrete, dtype: int64

Regression with continuous labels

In [ ]:
from sklearn.linear_model import linear_regression
#.6, .2, .2
y=labels.loc[y_label, 'label_continuous']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) 